# Getting Toronto data from wikipedia II
## Adding longitude and latitude coordinates

As the Toronto data that we need to explore, segment, and cluster the neighborhoods in the city of Toronto is not readily available, we need to scrape the Wikipedia page ([Postal_codes_of_Canada](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M)) and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format ready for further use. 

We set out to use the BeautifulSoup package so as to transform the data in the table on the Wikipedia page into pandas dataframe that is convenient for our demand

In [35]:
from bs4 import BeautifulSoup
import requests

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r=requests.get(url)
url=r.content
soup = BeautifulSoup(url,'html.parser')

table = soup.find("table",{"class":"wikitable sortable"})
#table

I now need to do an iteration through each row (tr element) and then assign each element in the tr to a variable. At this stage, I will grab everything in the Postcode, Borough, and Neighbourhood column. To do this, I used the following code:

In [36]:
for row in table.find_all("tr"):
    cells = row.find_all("td")
    #For each "tr", assign each "td" to a variable.
    if len(cells) == 3:
        PostalCode = cells[0].findAll(text=True)
        Borough = cells[1].findAll(text=True)
        Neighbourhood = cells[2].findAll(text=True)
        

After this code executes, I have a value for the Postcode, Borough, and Neighbourhood. Now in the second part of my iteration, I create a dictionary and at the end form a data frame, i.e convert the wikipedia table to pandas dataframe.

In [37]:
import pandas as pd
PostalCode=[]
Borough=[]
Neighbourhood=[]

for row in table.find_all('tr')[1:]:#to skip a header row
    col=row.find_all('td')
    PostalCode.append(col[0].text.strip())
    Borough.append(col[1].text.strip())
    Neighbourhood.append(col[2].text.strip())

columns={'PostalCode':PostalCode,'Borough':Borough,'Neighbourhood':Neighbourhood}
# Create a dataframe from the columns variable
Toronto_df = pd.DataFrame(columns)
Toronto_df.head(12)



,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [38]:
Toronto_df.shape

(289, 3)

Now let me drop rows with a Borough that is Not assigned

In [39]:
Toronto_df.drop(Toronto_df[Toronto_df.Borough == "Not assigned"].index, inplace=True)
Toronto_df.reset_index
Toronto_df.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [40]:
Toronto_df.shape

(212, 3)

Lets us combine those neighborhood belonging to the same postal code area

In [41]:
Toronto_df.set_index(['PostalCode','Borough'],inplace=True)
Toronto_df1 = Toronto_df.groupby(level=['PostalCode','Borough'], sort=False).agg( ','.join)
Toronto_df1.reset_index().head(12) # to remove set index

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Not assigned
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


In cases where a cell has a Borough but a 'Not assigned' Neighborhood, then the Neighborhood will be the same as the Borough. We use the following line of code to achieve that. 

In [42]:
Toronto_df1.loc[Toronto_df1['Neighbourhood'] =='Not assigned']=Toronto_df1[Toronto_df1['Neighbourhood'] == 'Not assigned'].index.values[0][1]
Toronto_df1.head(12)

,,Neighbourhood
PostalCode,Borough,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Harbourfront,Regent Park"
M6A,North York,"Lawrence Heights,Lawrence Manor"
M7A,Queen's Park,Queen's Park
M9A,Etobicoke,Islington Avenue
M1B,Scarborough,"Rouge,Malvern"
M3B,North York,Don Mills North
M4B,East York,"Woodbine Gardens,Parkview Hill"


In [43]:
df_Toronto=Toronto_df1.reset_index()
df_Toronto.head(12)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


Let us now check what is the size of the clean dataframe. Recall the original size of our dataframe has 289 rows and 3 columns. Though we haven't dropped any column we reiterate that we have deleted rows with 'Not assigned' Borough and combined Negibourhood who share same PostalCode area. 

In [44]:
df_Toronto.shape

(103, 3)

We now have 103 rows and 3 columns.

## Let us add the coordinates

In [45]:
!pip install geocoder

print('geocoder has been installed before.')


distributed 1.21.8 requires msgpack, which is not installed.
geocoder has been installed before.


In [46]:
#Takes Long time so I have to import the coordinates from the  
"""
import geocoder # import geocoder
latitude = []
longitude = []
for postal_code in PostalCode:
    # initialize your variable to None
    lat_lng_coords = None
    
# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude.append(lat_lng_coords[0])
longitude.append(lat_lng_coords[1])

Toronto_dict = {'Postalcode':Postalcode, 'Borough':Borough, 'Neighbourhood':Neighbourhood,
              'Latitude': latitude, 'Longitude':longitude}
df_toronto1 = pd.DataFrame.from_dict(Toronto_dict)
df_toronto1.to_csv('Toronto_Data.csv')
df_toronto1.head(10)
"""
print("Takes too long")

Takes too long


While I try to run the above code, I realized it took a very long time so as an alternative I have imported a Geospatial_data from ([Geographical_coordinates_for_each_postalcode](https://cocl.us/Geospatial_data)) to extract the coordinates

In [47]:
link =("https://cocl.us/Geospatial_data")
Torontogeodata_df = pd.read_csv(link)
#chech if my dataframe and the geospatial data has same dimension
print(Torontogeodata_df.shape)
Torontogeodata_df.head(12)

(103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


Notice in my dataframe (i.e. *df_Toronto*) I used column name "PostalCode", but from the geospational data we note the name of the column is "Postal Code". Therefore before I merge these two dataframes I have to rename the column so that they both have same column name. 

In [48]:
Torontogeodata_df.rename(columns = {'Postal Code':'PostalCode'}, inplace = True)
Toronto_data = pd.merge(df_Toronto, Torontogeodata_df, on="PostalCode",how='left')
Toronto_data.head(12)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937


In [49]:
Toronto_data.shape

(103, 5)

Let us save the file for further use

In [50]:
Toronto_data.to_csv('Toronto_Data.csv')
#check if the file is saved correctly
#as we dont need pandas to set index we enforce index_col=0
#lets us display the first 12 rows
pd.read_csv("Toronto_Data.csv",index_col=0).head(12)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937


In [51]:
#reCheck if we still have all rows and columns
pd.read_csv("Toronto_Data.csv",index_col=0).shape

(103, 5)